In [2]:
import csv
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt


In [3]:
#u novi folder prebacuje slike za koje postoje anotacije u csv fajlu
filename = "mpii_human_pose.csv"

fields = []
rows = []
 
with open(filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
 
    fields = next(csvreader)
 
    for row in csvreader:
        rows.append(row)
 
for row in rows[:len(rows)]:
    name = row[1]
    if os.path.exists("images/"+name):
        os.replace("images/"+name, "images_extract/"+name)


In [ ]:
#ucitava inforacije o slikama koje sadrze vise od jedne osobe
file1 = open("images.txt","r")
b = []
lines = file1.readlines()
for i in range(len(lines)):
    lines[i] = lines[i].rstrip()
    #brise slike
    if os.path.exists("images_extract/" + lines[i]):
        os.remove( "images_extract/" + lines[i])


In [4]:
# csv fajl cita kao pandas dataframe
df = pd.read_csv(filename)

In [5]:
#izbacuje sve redove iz tabele koji sadrze indeks slike koja se nalazi u fajlu images.txt
df = df.drop(df[df['NAME'].isin(lines)].index)

NameError: name 'lines' is not defined

In [ ]:
df

In [ ]:
#konvertuje df u csv i cuva ga
x = df.to_csv('mpii_new.csv', index=False)

In [ ]:
filename = "mpii_new.csv"

fields = []
rows = []
 
with open(filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
 
    fields = next(csvreader)
 
    for row in csvreader:
        rows.append(row)
 
#Izdvajanje trening podataka
for row in rows[:8001]:
    name = row[1]
    if os.path.exists("images_extract/"+name):
        os.replace("images_extract/"+name, "images_training/"+name)

#Izdvajanje test podataka
for row in rows[8001:]:
    name = row[1]
    if os.path.exists("images_extract/"+name):
        os.replace("images_extract/"+name, "images_test/"+name)

In [9]:
output_path = 'heat_maps_test_plain'
path = 'images_test'

if not os.path.exists(output_path):
    os.makedirs(output_path)

brojac = 0 

for image in os.listdir(path):
    brojac += 1
    
    img = Image.open(os.path.join(path, image))
    mapa = np.zeros((17, 368, 368))

    rezultat = df[df['NAME'] == image]
    rezultat_numpy = rezultat.to_numpy()[0]

    
    boja = 1
    sirina = 10
    sloj_brojac = -1

    for k in range(2, len(rezultat_numpy) - 3, 2):
        sloj_brojac += 1
        if rezultat_numpy[k] == -1 or rezultat_numpy[k + 1] == -1:
            continue

        x = round(rezultat_numpy[k] * 368 / img.size[0])
        y = round(rezultat_numpy[k + 1] * 368 / img.size[1])

        for i in range(max(0, y - sirina), min(mapa.shape[1], y + sirina)):
            for j in range(max(0, x - sirina), min(mapa.shape[2], x + sirina)):
                mapa[sloj_brojac, i, j] = boja

    
    for k in range(16):
        mapa[16] += mapa[k]
    mapa[16] = (16 - mapa[16]) / 16

    
    loc = os.path.join(output_path, 'hm_' + image.replace('.jpg', '.npy'))
    np.save(loc, mapa)
    
    print(brojac)  

1


In [ ]:
output_path = 'heat_maps_training_plain'
path = 'images_training'

if not os.path.exists(output_path):
    os.makedirs(output_path)

brojac = 0 

for image in os.listdir(path):
    brojac += 1
    
    img = Image.open(os.path.join(path, image))
    mapa = np.zeros((17, 368, 368))

    rezultat = df[df['NAME'] == image]
    rezultat_numpy = rezultat.to_numpy()[0]

    
    boja = 1
    sirina = 10
    sloj_brojac = -1

    for k in range(2, len(rezultat_numpy) - 3, 2):
        sloj_brojac += 1
        if rezultat_numpy[k] == -1 or rezultat_numpy[k + 1] == -1:
            continue

        x = round(rezultat_numpy[k] * 368 / img.size[0])
        y = round(rezultat_numpy[k + 1] * 368 / img.size[1])

        for i in range(max(0, y - sirina), min(mapa.shape[1], y + sirina)):
            for j in range(max(0, x - sirina), min(mapa.shape[2], x + sirina)):
                mapa[sloj_brojac, i, j] = boja

    
    for k in range(16):
        mapa[16] += mapa[k]
    mapa[16] = (16 - mapa[16]) / 16

    
    loc = os.path.join(output_path, 'hm_' + image.replace('.jpg', '.npy'))
    np.save(loc, mapa)
    
    print(brojac)  